In [ ]:
import numpy as np
import pandas as pd
import re
import pyarrow.feather as feather
import random
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Train

**Train**
Таргет для обучения с оценками и количеством долгов студентов за 2017 - 2020 учебные годы (без 2021/2022 года)


ISU - UID студента

SEMESTER - семестр получения оценки

DISC_ID - UID дисциплины

DEBT - факт наличия долга (0 - отсутствие, 1 - наличие) - таргет

In [ ]:
train = pd.read_csv(
    '/content/drive/MyDrive/Colab Notebooks/Pet_project/DF/train.csv')
train[:3]

,ISU,ST_YEAR,SEMESTER,DISC_ID,TYPE_NAME,DEBT
0,17623928044460629977,2020,1,10502311854018326223,Зачет,0
1,17623928044460629977,2020,1,1601392918367593206,Зачет,0
2,17623928044460629977,2020,1,9559803959325174929,Зачет,0


In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 178899 entries, 0 to 178898
Data columns (total 6 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   ISU        178899 non-null  uint64
 1   ST_YEAR    178899 non-null  int64 
 2   SEMESTER   178899 non-null  int64 
 3   DISC_ID    178899 non-null  uint64
 4   TYPE_NAME  178899 non-null  object
 5   DEBT       178899 non-null  int64 
dtypes: int64(3), object(1), uint64(2)
memory usage: 8.2+ MB


In [ ]:
train.shape

(178899, 6)

# Marks

**Таблица comp_marks.csv**

ISU - UID студента

ST_YEAR - год начала курса (первое число из формата 2020/2021)

SEMESTER - фактический семестр, в котором получена оценка

TYPE_NAME - форма отчетности

MARK - оценка (или зачет/незачет)

MAIN_PLAN - учебный план

DISC_ID - UID дисциплины

PRED_ID - UID преподавателя

In [ ]:
marks = pd.read_csv(
    '/content/drive/MyDrive/Colab Notebooks/Pet_project/DF/comp_marks.csv')
marks[:3]

,ISU,ST_YEAR,SEMESTER,TYPE_NAME,MARK,MAIN_PLAN,DISC_ID,PRED_ID
0,17623928044460629977,2020,1,Зачет,зачет,4664712865904477750,10502311854018326223,5087927234000983963
1,17623928044460629977,2020,1,Зачет,зачет,4664712865904477750,1601392918367593206,5087927234000983963
2,17623928044460629977,2020,1,Зачет,зачет,4664712865904477750,9559803959325174929,5087927234000983963


In [ ]:
marks.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 182795 entries, 0 to 182794
Data columns (total 8 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   ISU        182795 non-null  uint64
 1   ST_YEAR    182795 non-null  int64 
 2   SEMESTER   182795 non-null  int64 
 3   TYPE_NAME  182795 non-null  object
 4   MARK       182208 non-null  object
 5   MAIN_PLAN  182795 non-null  uint64
 6   DISC_ID    182795 non-null  uint64
 7   PRED_ID    182795 non-null  uint64
dtypes: int64(2), object(2), uint64(4)
memory usage: 11.2+ MB


In [ ]:
marks.shape

(182795, 8)

In [ ]:
marks['MARK'].unique()

array(['зачет', '3', '4', 'незач', '5', '2', 'осв', nan, 'неявка'],
      dtype=object)

In [ ]:
# Зполним пропуски значением 'неявка'
marks = marks.fillna('неявка')

# Discipline

**Таблица comp_disc.csv**

PLAN_ID - UID учебного плана (то же что и MAIN_PLAN - значения в этих колонках соответствуют друг другу и могут быть использованы для сопоставления таблиц)

DISC_ID - UID дисциплины

СHOICE - выборность дисциплины

SEMESTER - семестр проведения дисциплины в данном учебном плане

DISC_NAME - название дисциплины

DISC_DEP - факультет-реализатор дисциплины

In [ ]:
discipline = pd.read_csv(
    '/content/drive/MyDrive/Colab Notebooks/Pet_project/DF/comp_disc.csv')
discipline[:3]

,PLAN_ID,DISC_ID,СHOICE,SEMESTER,DISC_NAME,DISC_DEP,KEYWORD_NAMES
0,13019929768422386211,12540082885330665258,1.0,6.0,Методы криптографии,8139328495281102994,"криптография, криптография, криптография, клас..."
1,13019929768422386211,8471500331110227546,1.0,6.0,Анализ социальных сетей,8139328495281102994,"технологии работы информацией, алгоритмы обраб..."
2,13019929768422386211,11786710099399385350,1.0,6.0,Эффективное участие в научных и практических к...,1687590892619214362,NaN


In [ ]:
discipline.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22517 entries, 0 to 22516
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   PLAN_ID        22517 non-null  uint64 
 1   DISC_ID        22517 non-null  uint64 
 2   СHOICE         22517 non-null  float64
 3   SEMESTER       22517 non-null  float64
 4   DISC_NAME      22517 non-null  object 
 5   DISC_DEP       22517 non-null  uint64 
 6   KEYWORD_NAMES  13759 non-null  object 
dtypes: float64(2), object(2), uint64(3)
memory usage: 1.2+ MB


In [ ]:
discipline.shape

(22517, 7)

In [ ]:
# удалим колонку KEYWORD_NAMES
del discipline['KEYWORD_NAMES']

# поменяем тип колонки SEMESTER на int
discipline['SEMESTER'] = discipline['SEMESTER'].astype('int16')

# переименуем колонку PLAN_ID для последующей конкатенации
discipline.rename(columns={'PLAN_ID': 'MAIN_PLAN'}, inplace=True)
discipline['DISC_DEP'] = discipline['DISC_DEP'].astype('str')

# соединим Marks и Discipline
temp = marks.merge(discipline, on=['MAIN_PLAN', 'DISC_ID', 'SEMESTER'], how='left')

In [ ]:
temp[:3]

,ISU,ST_YEAR,SEMESTER,TYPE_NAME,MARK,MAIN_PLAN,DISC_ID,PRED_ID,СHOICE,DISC_NAME,DISC_DEP
0,17623928044460629977,2020,1,Зачет,зачет,4664712865904477750,10502311854018326223,5087927234000983963,0.0,Введение в профессиональную деятельность,5077682191738720383
1,17623928044460629977,2020,1,Зачет,зачет,4664712865904477750,1601392918367593206,5087927234000983963,0.0,Введение в цифровую культуру,8139328495281102994
2,17623928044460629977,2020,1,Зачет,зачет,4664712865904477750,9559803959325174929,5087927234000983963,0.0,Дискретная математика,5077682191738720383


In [ ]:
temp.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 264493 entries, 0 to 264492
Data columns (total 11 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   ISU        264493 non-null  uint64 
 1   ST_YEAR    264493 non-null  int64  
 2   SEMESTER   264493 non-null  int64  
 3   TYPE_NAME  264493 non-null  object 
 4   MARK       264493 non-null  object 
 5   MAIN_PLAN  264493 non-null  uint64 
 6   DISC_ID    264493 non-null  uint64 
 7   PRED_ID    264493 non-null  uint64 
 8   СHOICE     264486 non-null  float64
 9   DISC_NAME  264486 non-null  object 
 10  DISC_DEP   264486 non-null  object 
dtypes: float64(1), int64(2), object(4), uint64(4)
memory usage: 24.2+ MB


# Teachers

**Таблица comp_teachers.csv**

ISU - UID преподавателя

GENDER - Пол

DATE_BIRTH - Год рождения преподавателя

ST_YEAR - год обучения

SEMESTER - семестр преподавания

DISC_ID - UID преподаваемой дисциплины

MAIN_PLAN - UID учебного плана данной программы

TYPE_NAME - форма контроля по предмету (экзамен, зачет, дифференцированный зачет)

MARK - средняя оценка, выставленная преподавателем

In [ ]:
teachers = pd.read_csv(
    '/content/drive/MyDrive/Colab Notebooks/Pet_project/DF/comp_teachers.csv')
teachers[:3]

,ISU,GENDER,DATE_BIRTH,ST_YEAR,SEMESTER,DISC_ID,MAIN_PLAN,TYPE_NAME,MARK
0,10629403761260861411,М,1975,2020/2021,4.0,13112935959748478559,10080731777079285074,Дифференцированный зачет,3.471698
1,10629403761260861411,М,1975,2018/2019,2.0,9080510863460073035,15820306466107805201,Дифференцированный зачет,4.243902
2,10629403761260861411,М,1975,2019/2020,4.0,5019820477479850617,3636112309748229871,Экзамен,4.652174


In [ ]:
teachers.shape

(5554, 9)

In [ ]:
teachers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5554 entries, 0 to 5553
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   ISU         5554 non-null   uint64 
 1   GENDER      5554 non-null   object 
 2   DATE_BIRTH  5554 non-null   int64  
 3   ST_YEAR     5554 non-null   object 
 4   SEMESTER    5334 non-null   float64
 5   DISC_ID     5554 non-null   uint64 
 6   MAIN_PLAN   5554 non-null   uint64 
 7   TYPE_NAME   5334 non-null   object 
 8   MARK        3443 non-null   float64
dtypes: float64(2), int64(1), object(3), uint64(3)
memory usage: 390.6+ KB


In [ ]:
# Проверим совпадают ли ID преподавателей в наших таблицах
temp['PRED_ID'].isin(teachers['ISU'].values).unique()

array([False])

**у нас ID преподавателей в таблицах не совпадают, тогда возьмем из нее только средние оценки по каждому предмету. В таблице есть попуски в оценках, разберемся с ними** 

In [ ]:
teachers[teachers['SEMESTER'].isna()]

,ISU,GENDER,DATE_BIRTH,ST_YEAR,SEMESTER,DISC_ID,MAIN_PLAN,TYPE_NAME,MARK
244,16835290750631533220,М,1960,2019/2020,NaN,695931230947311728,8417928891314317457,NaN,NaN
324,3536591057520059320,М,1981,2019/2020,NaN,1607124926567219753,1486340937557991862,NaN,NaN
484,13044226354785628353,Ж,1968,2020/2021,NaN,3349554631096550219,4862064472418713298,NaN,NaN
514,2755304833628407074,М,1941,2019/2020,NaN,17249310526307120924,8417928891314317457,NaN,NaN
526,14797593412921377792,Ж,1970,2020/2021,NaN,5683968764673783541,6252077101057070041,NaN,NaN
...,...,...,...,...,...,...,...,...,...
5548,11372188095720335175,М,1986,2020/2021,NaN,7766831654069050069,12802389387899367182,NaN,NaN
5549,11372188095720335175,М,1986,2020/2021,NaN,7766831654069050069,2580376221820191753,NaN,NaN
5550,11372188095720335175,М,1986,2020/2021,NaN,7766831654069050069,7130526432801337743,NaN,NaN
5552,11372188095720335175,М,1986,2020/2021,NaN,5066897240534486806,12572440150254943648,NaN,NaN


**Идея такая: Заполнить их средними оценками по этому предмету в семестре. Но у нас есть пропуски в семестрах. Их можно заполнить самым часто встречающимся значением семестра по этому предмету в каждом году**

In [ ]:
teachers['SEMESTER'].unique()

array([ 4.,  2.,  1.,  3.,  6.,  5., nan])

In [ ]:
teachers['ST_YEAR'].unique()

array(['2020/2021', '2018/2019', '2019/2020'], dtype=object)

In [ ]:
# Найдем индексы строк где значение семестра Nan
index = teachers['SEMESTER'].index[teachers['SEMESTER'].apply(np.isnan)]
index

Int64Index([ 244,  324,  484,  514,  526,  682,  685,  687,  921,  953,
            ...
            5525, 5526, 5528, 5529, 5532, 5548, 5549, 5550, 5552, 5553],
           dtype='int64', length=220)

In [ ]:
# заполним пропуски модой для каждого семестра
for i in index:
  teachers['SEMESTER'].iloc[i] = discipline.loc[
      discipline['DISC_ID']==discipline['DISC_ID'].iloc[i]]['SEMESTER'].mode().max()

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [ ]:
# Нам нужно колонку год заполнить однозначными значениями года. 
# Так как семестры у нас идут сквозным способом, то сопоставим года и семестры.
teachers['ST_YEAR'] = np.where(teachers['SEMESTER'] == 1, 
                               2018, 
                               np.where(teachers['SEMESTER'].isin([2.0, 3.0]), 
                                        2019, 
                                        np.where(teachers['SEMESTER'].isin([4.0, 5.0]), 
                                                 2020, 
                                                 2021)))

In [ ]:
teachers['ST_YEAR'].unique()

array([2020, 2019, 2018, 2021])

In [ ]:
teachers[:3]

,ISU,GENDER,DATE_BIRTH,ST_YEAR,SEMESTER,DISC_ID,MAIN_PLAN,TYPE_NAME,MARK
0,10629403761260861411,М,1975,2020,4.0,13112935959748478559,10080731777079285074,Дифференцированный зачет,3.471698
1,10629403761260861411,М,1975,2019,2.0,9080510863460073035,15820306466107805201,Дифференцированный зачет,4.243902
2,10629403761260861411,М,1975,2020,4.0,5019820477479850617,3636112309748229871,Экзамен,4.652174


In [ ]:
# теперь заполним пропуски в TYPE_NAME модой по дисциплинам. Найдем индексы Nan, пройдем по ним 
# и заполним
index = teachers['TYPE_NAME'].index[teachers['TYPE_NAME'].isna()]
index

Int64Index([ 244,  324,  484,  514,  526,  682,  685,  687,  921,  953,
            ...
            5525, 5526, 5528, 5529, 5532, 5548, 5549, 5550, 5552, 5553],
           dtype='int64', length=220)

In [ ]:
for i in index:
  teachers['TYPE_NAME'].iloc[i] = teachers[
      teachers['DISC_ID'] == teachers['DISC_ID'].iloc[i]]['TYPE_NAME'].mode().max()

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [ ]:
teachers['TYPE_NAME'].fillna(teachers['TYPE_NAME'].mode().max(), inplace=True)

In [ ]:
# Заменим странный зачет на просто зачет.
teachers['TYPE_NAME'] = teachers.apply(
    lambda x: re.sub('Странный зачет', 'Зачет', x['TYPE_NAME']), axis=1)

In [ ]:
# ну и заполним оценки средними значениями по дисциплинам
index = teachers['MARK'].index[teachers['MARK'].apply(np.isnan)]
index

Int64Index([   5,    8,   21,   22,   31,   34,   35,   38,   39,   44,
            ...
            5534, 5535, 5536, 5543, 5544, 5548, 5549, 5550, 5552, 5553],
           dtype='int64', length=2111)

In [ ]:
# поменяем тип колонки DISC_ID на str
teachers['DISC_ID'] = teachers['DISC_ID'].astype('str')

In [ ]:
teachers[teachers['MARK'].isna()]

,ISU,GENDER,DATE_BIRTH,ST_YEAR,SEMESTER,DISC_ID,MAIN_PLAN,TYPE_NAME,MARK
5,10629403761260861411,М,1975,2018,1.0,2604062592206042930,3636112309748229871,Зачет,NaN
8,10629403761260861411,М,1975,2018,1.0,8210405244786072071,3259025934245756351,Зачет,NaN
21,6674699098929009726,М,1975,2018,1.0,6673786453515598761,11623170453297887849,Зачет,NaN
22,6674699098929009726,М,1975,2018,1.0,6673786453515598761,15970149812698605927,Зачет,NaN
31,9518828187737345193,М,1972,2018,1.0,2217818214435684685,4833027042617058934,Зачет,NaN
...,...,...,...,...,...,...,...,...,...
5548,11372188095720335175,М,1986,2019,3.0,7766831654069050069,12802389387899367182,Экзамен,NaN
5549,11372188095720335175,М,1986,2019,3.0,7766831654069050069,2580376221820191753,Экзамен,NaN
5550,11372188095720335175,М,1986,2020,5.0,7766831654069050069,7130526432801337743,Экзамен,NaN
5552,11372188095720335175,М,1986,2018,1.0,5066897240534486806,12572440150254943648,Экзамен,NaN


In [ ]:
# Найдем индексы Nan, пройдем по ним и заполним
index = teachers['MARK'].index[teachers['MARK'].apply(np.isnan)]
index

Int64Index([   5,    8,   21,   22,   31,   34,   35,   38,   39,   44,
            ...
            5534, 5535, 5536, 5543, 5544, 5548, 5549, 5550, 5552, 5553],
           dtype='int64', length=2111)

In [ ]:
for i in index:
  teachers['MARK'].iloc[i] = teachers[
      teachers['TYPE_NAME'] == teachers['TYPE_NAME'].iloc[i]]['MARK'].mean()

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [ ]:
teachers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5554 entries, 0 to 5553
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   ISU         5554 non-null   uint64 
 1   GENDER      5554 non-null   object 
 2   DATE_BIRTH  5554 non-null   int64  
 3   ST_YEAR     5554 non-null   int64  
 4   SEMESTER    5554 non-null   float64
 5   DISC_ID     5554 non-null   object 
 6   MAIN_PLAN   5554 non-null   uint64 
 7   TYPE_NAME   5554 non-null   object 
 8   MARK        5554 non-null   float64
dtypes: float64(2), int64(2), object(3), uint64(2)
memory usage: 390.6+ KB


In [ ]:
# изменим тип данных у семестра, так как по факту это натуральные числа.
teachers['SEMESTER'] = teachers['SEMESTER'].astype('int16')

In [ ]:
temp[:3]

,ISU,ST_YEAR,SEMESTER,TYPE_NAME,MARK,MAIN_PLAN,DISC_ID,PRED_ID,СHOICE,DISC_NAME,DISC_DEP
0,17623928044460629977,2020,1,Зачет,зачет,4664712865904477750,10502311854018326223,5087927234000983963,0.0,Введение в профессиональную деятельность,5077682191738720383
1,17623928044460629977,2020,1,Зачет,зачет,4664712865904477750,1601392918367593206,5087927234000983963,0.0,Введение в цифровую культуру,8139328495281102994
2,17623928044460629977,2020,1,Зачет,зачет,4664712865904477750,9559803959325174929,5087927234000983963,0.0,Дискретная математика,5077682191738720383


In [ ]:
teachers[:3]

,ISU,GENDER,DATE_BIRTH,ST_YEAR,SEMESTER,DISC_ID,MAIN_PLAN,TYPE_NAME,MARK
0,10629403761260861411,М,1975,2020,4,13112935959748478559,10080731777079285074,Дифференцированный зачет,3.471698
1,10629403761260861411,М,1975,2019,2,9080510863460073035,15820306466107805201,Дифференцированный зачет,4.243902
2,10629403761260861411,М,1975,2020,4,5019820477479850617,3636112309748229871,Экзамен,4.652174


In [ ]:
# переименуем колонку MARK иначе при конкатенации будет дубль.
teachers.rename(columns={'MARK': 'mean_MARK'}, inplace=True)

In [ ]:
# изменим тип DISC_ID чтобы объединить данные
teachers['DISC_ID'] = teachers['DISC_ID'].astype('uint64')

In [ ]:
# добавим колонку с оценками
temp = temp.merge(teachers.groupby(['DISC_ID'])['mean_MARK'].mean(), 
                  on=['DISC_ID'], 
                  how='left')

In [ ]:
# остаток пропусков в mean_MARK заполним средними.
temp.fillna(temp['mean_MARK'].mean(), inplace=True)

In [ ]:
temp.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 264493 entries, 0 to 264492
Data columns (total 12 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   ISU        264493 non-null  uint64 
 1   ST_YEAR    264493 non-null  int64  
 2   SEMESTER   264493 non-null  int64  
 3   TYPE_NAME  264493 non-null  object 
 4   MARK       264493 non-null  object 
 5   MAIN_PLAN  264493 non-null  uint64 
 6   DISC_ID    264493 non-null  uint64 
 7   PRED_ID    264493 non-null  uint64 
 8   СHOICE     264493 non-null  float64
 9   DISC_NAME  264493 non-null  object 
 10  DISC_DEP   264493 non-null  object 
 11  mean_MARK  264493 non-null  float64
dtypes: float64(2), int64(2), object(4), uint64(4)
memory usage: 26.2+ MB


# Portrait

**Таблица comp_portrait.csv ~ 5000 строк**

ISU - UID студента

GENDER - пол

CITIZENSHIP - гражданство

EXAM_TYPE - форма зачисления (ЕГЭ, олимпиада, ВИ - вступительные испытания)

EXAM_SUBJECT_1 - первый экзамен ЕГЭ

EXAM_SUBJECT_2 - второй экзамен ЕГЭ

EXAM_SUBJECT_3 - третий экзамен ЕГЭ

ADMITTED_EXAM_1 - баллы за 1 экзамен ЕГЭ

ADMITTED_EXAM_2 - баллы за 2 экзамен ЕГЭ

ADMITTED_EXAM_3 - баллы за 3 экзамен ЕГЭ

ADMITTED_SUBJECT_PRIZE_LEVEL - уровень олимпиады (если есть)

REGION_ID - номер региона

In [ ]:
portrait = pd.read_csv(
    '/content/drive/MyDrive/Colab Notebooks/Pet_project/DF/comp_portrait.csv')
portrait[:3]

,ISU,GENDER,CITIZENSHIP,EXAM_TYPE,EXAM_SUBJECT_1,EXAM_SUBJECT_2,EXAM_SUBJECT_3,ADMITTED_EXAM_1,ADMITTED_EXAM_2,ADMITTED_EXAM_3,ADMITTED_SUBJECT_PRIZE_LEVEL,REGION_ID
0,8746993777733478780,Ж,15601729049989747827,ЕГЭ,70786669040476600,5533732657842394915,8388269026169219461,88.0,91.0,81.0,ЕГЭ,4877310761925081124
1,18279003839105282485,М,15601729049989747827,ВИ,70786669040476600,5533732657842394915,8388269026169219461,84.0,89.0,96.0,ЕГЭ,4877310761925081124
2,17601399077733331421,М,15601729049989747827,ВИ,70786669040476600,5533732657842394915,8388269026169219461,66.0,78.0,93.0,ЕГЭ,4877310761925081124


In [ ]:
portrait.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8583 entries, 0 to 8582
Data columns (total 12 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   ISU                           8583 non-null   uint64 
 1   GENDER                        8583 non-null   object 
 2   CITIZENSHIP                   8583 non-null   uint64 
 3   EXAM_TYPE                     8582 non-null   object 
 4   EXAM_SUBJECT_1                8583 non-null   uint64 
 5   EXAM_SUBJECT_2                8583 non-null   uint64 
 6   EXAM_SUBJECT_3                8583 non-null   uint64 
 7   ADMITTED_EXAM_1               7404 non-null   float64
 8   ADMITTED_EXAM_2               7404 non-null   float64
 9   ADMITTED_EXAM_3               7404 non-null   float64
 10  ADMITTED_SUBJECT_PRIZE_LEVEL  8583 non-null   object 
 11  REGION_ID                     8583 non-null   uint64 
dtypes: float64(3), object(3), uint64(6)
memory usage: 804.8+ KB


In [ ]:
portrait.shape

(8583, 12)

In [ ]:
# Посмотрм на строки с пропущенными значениями
portrait[portrait.isnull().any(1)]

,ISU,GENDER,CITIZENSHIP,EXAM_TYPE,EXAM_SUBJECT_1,EXAM_SUBJECT_2,EXAM_SUBJECT_3,ADMITTED_EXAM_1,ADMITTED_EXAM_2,ADMITTED_EXAM_3,ADMITTED_SUBJECT_PRIZE_LEVEL,REGION_ID
50,15482195001482259050,М,15601729049989747827,ОЛИМПИАДА,70786669040476600,8388269026169219461,5533732657842394915,NaN,NaN,NaN,1.0,4877310761925081124
52,17668820379502778901,М,15601729049989747827,ОЛИМПИАДА,70786669040476600,5533732657842394915,8388269026169219461,NaN,NaN,NaN,1.0,4877310761925081124
59,14474819382042716420,М,15601729049989747827,ОЛИМПИАДА,70786669040476600,5533732657842394915,8388269026169219461,NaN,NaN,NaN,1.0,1109768648583698762
64,894502649105671518,М,15601729049989747827,ОЛИМПИАДА,70786669040476600,8388269026169219461,14462659375807213512,NaN,NaN,NaN,3.0,14816259123945808471
293,1161659277259750342,М,15601729049989747827,ОЛИМПИАДА,70786669040476600,5533732657842394915,8388269026169219461,NaN,NaN,NaN,1.0,4877310761925081124
...,...,...,...,...,...,...,...,...,...,...,...,...
8511,13952650986396392717,М,15601729049989747827,ОЛИМПИАДА,5533732657842394915,70786669040476600,8388269026169219461,NaN,NaN,NaN,1.0,8611857524221992249
8512,17923172498722874079,М,15601729049989747827,ОЛИМПИАДА,5533732657842394915,70786669040476600,8388269026169219461,NaN,NaN,NaN,2.0,4877310761925081124
8513,6611550158179331872,М,15601729049989747827,ОЛИМПИАДА,5533732657842394915,70786669040476600,8388269026169219461,NaN,NaN,NaN,2.0,698269158467562988
8578,10617950247642298995,М,15601729049989747827,ОЛИМПИАДА,70786669040476600,8388269026169219461,5533732657842394915,NaN,NaN,NaN,1.0,1360429390938723525


**Видно, что некоторые студенты не имеют информации об оценках на экзаменах. Маловероятно, что они набрали 0 баллов, поэтому заполним данные средними значениями.**

In [ ]:
# Зполняем средними, меняем тип данных ISU на str и переименовываем колонки для
# конкатенации.
for i in range(1, 4):
  portrait['ADMITTED_EXAM_' + str(i)] = portrait[
      'ADMITTED_EXAM_' + str(i)].fillna(portrait['ADMITTED_EXAM_' + str(i)].mean())

portrait.dropna(inplace=True)

for i in portrait.drop('ISU', axis=1).select_dtypes(include='uint64'):
    portrait[i] = portrait[i].astype('str')

portrait.rename(columns={"GENDER":"STUD_GENDER"}, inplace=True)

temp = temp.merge(portrait, on=['ISU'], how='left')

# Students

**Таблица comp_students.csv ~11000 строк**

ISU - UID студента

COURSE - курс обучения, которому соответствует запись

DATE_START - начало периода обучения

DATE_END - конец периода обучения

PRIZNAK - состояние на конец периода обучения (обучен, академ, отчислен)

MAIN_PLAN - образовательный план в данный промежуток времени

In [ ]:
students = pd.read_csv(
    '/content/drive/MyDrive/Colab Notebooks/Pet_project/DF/comp_students.csv')
students[:3]

,ISU,KURS,DATE_START,DATE_END,PRIZNAK,MAIN_PLAN
0,4111128904589594590,2,2017-09-01,2018-08-31 00:00:00,обучен,1888061104212561891
1,4111128904589594590,3,2018-09-01,2018-11-02 00:00:00,обучен,1888061104212561891
2,4111128904589594590,3,2018-11-03,2019-08-31 00:00:00,обучен,1888061104212561891


In [ ]:
students.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20586 entries, 0 to 20585
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   ISU         20586 non-null  uint64
 1   KURS        20586 non-null  int64 
 2   DATE_START  20586 non-null  object
 3   DATE_END    20586 non-null  object
 4   PRIZNAK     19029 non-null  object
 5   MAIN_PLAN   20586 non-null  uint64
dtypes: int64(1), object(3), uint64(2)
memory usage: 965.1+ KB


In [ ]:
students.shape

(20586, 6)

In [ ]:
# Есть пропуски в PRIZNAK
students['PRIZNAK'].unique()

array(['обучен', 'отчисл', 'академ', nan], dtype=object)

In [ ]:
# Удалим такие строки
students.dropna(inplace=True)

# Добавим признак срок обучения(Training_period), который мы можем вычислить из даты начала и 
# даты окончания. После чего удалим дату кончания и переименуем колонку.
students['DATE_START'] = pd.to_datetime(students['DATE_START'])
students['DATE_END'] = pd.to_datetime(students['DATE_END'])
students['Training_period'] = (students['DATE_END'] - students['DATE_START']).dt.days
students.drop('DATE_END', axis=1, inplace=True)
students['DATE_START'] = students['DATE_START'].dt.year
students.rename(columns={'DATE_START': 'ST_YEAR'}, inplace=True)

students.drop('MAIN_PLAN', axis=1, inplace=True)
temp = temp.merge(students, on=['ISU', 'ST_YEAR'], how='left')

In [ ]:
# удалим оставшиеся пропуски
temp.dropna(inplace=True)

In [ ]:
# соединим train и temp, предварительно переведя типы uint64 в str.
columns_to_join = train.columns[train.columns.isin(temp.columns)].to_list()

for i in train.select_dtypes(include='uint64'):
    train[i] = train[i].astype('str')

for i in temp.select_dtypes(include='uint64'):
    temp[i] = temp[i].astype('str')

train = train.merge(temp,
                  on=columns_to_join,
                  how='left')
train.dropna(inplace=True)

In [ ]:
# оптимизируем типы данных нашего датасета
train[train.select_dtypes('int64').columns] = train[train.select_dtypes(
    'int64').columns].astype('int16')
train[train.select_dtypes('float64').columns] = train[train.select_dtypes(
    'float64').columns].astype('int16')
train[train.select_dtypes('uint64').columns] = train[train.select_dtypes(
    'uint64').columns].astype('str')
train[train.select_dtypes('object').columns] = train[train.select_dtypes(
    'object').columns].astype('str')

In [ ]:
train[:3]

,ISU,ST_YEAR,SEMESTER,DISC_ID,TYPE_NAME,DEBT,MARK,MAIN_PLAN,PRED_ID,СHOICE,...,EXAM_SUBJECT_2,EXAM_SUBJECT_3,ADMITTED_EXAM_1,ADMITTED_EXAM_2,ADMITTED_EXAM_3,ADMITTED_SUBJECT_PRIZE_LEVEL,REGION_ID,KURS,PRIZNAK,Training_period
0,17623928044460629977,2020,1,10502311854018326223,Зачет,0,зачет,4664712865904477750,5087927234000983963,0,...,5533732657842394915,8388269026169219461,78,79,91,ЕГЭ,7805492244297918082,2,обучен,65
1,17623928044460629977,2020,1,10502311854018326223,Зачет,0,зачет,4664712865904477750,5087927234000983963,0,...,5533732657842394915,8388269026169219461,78,79,91,ЕГЭ,7805492244297918082,2,академ,298
2,17623928044460629977,2020,1,1601392918367593206,Зачет,0,зачет,4664712865904477750,5087927234000983963,0,...,5533732657842394915,8388269026169219461,78,79,91,ЕГЭ,7805492244297918082,2,обучен,65


In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 357448 entries, 0 to 361449
Data columns (total 27 columns):
 #   Column                        Non-Null Count   Dtype 
---  ------                        --------------   ----- 
 0   ISU                           357448 non-null  object
 1   ST_YEAR                       357448 non-null  int16 
 2   SEMESTER                      357448 non-null  int16 
 3   DISC_ID                       357448 non-null  object
 4   TYPE_NAME                     357448 non-null  object
 5   DEBT                          357448 non-null  int16 
 6   MARK                          357448 non-null  object
 7   MAIN_PLAN                     357448 non-null  object
 8   PRED_ID                       357448 non-null  object
 9   СHOICE                        357448 non-null  int16 
 10  DISC_NAME                     357448 non-null  object
 11  DISC_DEP                      357448 non-null  object
 12  mean_MARK                     357448 non-null  int16 
 13 

# Save Data

In [ ]:
feather.write_feather(
    train, '/content/drive/MyDrive/Colab Notebooks/Pet_project/Processed data/train.feather')